In [1]:
import glob
import os

import scipy.io as sio

import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

from misc.patch_extractor import PatchExtractor
from misc.utils import rm_n_mkdir

from config import Config

In [51]:
cfg = Config()

extract_type = 'mirror' # 'valid' for fcn8 segnet etc.
                        # 'mirror' for u-net etc.
# check the patch_extractor.py 'main' to see the different

# orignal size (win size) - input size - output size (step size)
# 512x512 - 256x256 - 256x256 fcn8, dcan, segnet
# 536x536 - 268x268 - 84x84   unet, dist
# 540x540 - 270x270 - 80x80   xy, hover
# 504x504 - 252x252 - 252x252 micronetcd tr
step_size = [160, 160] # should match self.train_mask_shape (config.py) 
win_size  = [400, 400] # should be at least twice time larger than 
                       # self.train_base_shape (config.py) to reduce 
                       # the padding effect during augmentation

xtractor = PatchExtractor(win_size, step_size)

out_dir = "/home1/gzy/NucleiSegmentation/PanNuke/Test/%dx%d_%dx%d" % \
                    (win_size[0], win_size[1], step_size[0], step_size[1])
rm_n_mkdir(out_dir)

In [52]:
fold = 'fold3'
masks = np.load("/home1/gzy/NucleiSegmentation/PanNuke/Fold3/masks/%s/masks.npy" % fold)
types = np.load("/home1/gzy/NucleiSegmentation/PanNuke/Fold3/images/%s/types.npy" % fold)
images = np.load("/home1/gzy/NucleiSegmentation/PanNuke/Fold3/images/%s/images.npy" % fold)

In [53]:
for i in tqdm(range(images.shape[0])):
    img = images[i]
    img = img.astype('int32')
    mask = masks[i]
    mask = mask.reshape(256*256, 6)
    
    try:
        mask_type = [np.where(r!=0)[0][0] for r in mask]
    except Exception:
        print(i)
        continue
    mask_type = np.array(mask_type)
    mask_type = mask_type.reshape(256,256)
    # bg 为 0
    mask_type = mask_type + 1
    mask_type[mask_type == 6] = 0
    
    mask_inst = np.sum(mask, axis=1).reshape(256,256)
    mask_inst[mask_type==0] = 0
    
    ann = np.dstack([mask_inst, mask_type])
    ann = ann.astype('int32')
    img = np.concatenate([img, ann], axis=-1)
    sub_patches = xtractor.extract(img, extract_type)
    for idx, patch in enumerate(sub_patches):
        np.save("{0}/{1}_{2:03d}.npy".format(out_dir, str(i)+fold, idx), patch)

 50%|████▉     | 1356/2722 [07:09<06:55,  3.29it/s]

1356


 55%|█████▌    | 1506/2722 [07:57<06:34,  3.08it/s]

1506
1507
1508
1509
1510
1511


 58%|█████▊    | 1566/2722 [08:14<05:59,  3.22it/s]

...

 73%|███████▎  | 1987/2722 [10:11<04:05,  2.99it/s]

1987
1988
1989
1990
1991
1992


 76%|███████▌  | 2063/2722 [10:34<03:57,  2.77it/s]

2063
2064
2065
2066
2067


100%|██████████| 2722/2722 [14:03<00:00,  3.23it/s]
